In [ ]:
'''plot syle setting'''
from jupyterthemes import jtplot
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
jtplot.style('onedork')#('grade3')

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# '''use modin as pandas, using dask engine rather than ray'''
# from distributed import Client
# dask_client = Client()
# os.environ["MODIN_ENGINE"] = "dask"
# import modin.pandas as pd

# Preprocessing data
IllustrisTNG https://www.tng-project.org/data/downloads/TNG50-2/   
> Here, we use TNG-50-2 datasets   
> - TNG-50-2 full hydro simulation   
> - TNG-50-2-dark is simple dark matter simulation   
> path: ~/data/mydata/TNG/TNG50   

We wanna use **V-Net** structure.
> for the first simple test, wo only need the z=10 to z=0 velocity filed or density field   
> then we need to use INPUT (the same properties with fastPM) to get OUTPUT (subhalo properties)

## load `TNG-50` data

In [15]:
'''import illustris_python package   
https://www.tng-project.org/data/docs/scripts/
'''
import os
HOME = os.path.expandvars('$HOME')
illustris_python_path = HOME+'/data/mydata/TNG'
if illustris_python_path not in os.sys.path:
    os.sys.path.append(illustris_python_path)
#     os.sys.path.insert(0,illustris_python_path)
# print(f'Our python package path is \n{os.sys.path}')
import illustris_python as ill

In [16]:
# TNG-50-2-dark
basePath = HOME + '/data/mydata/TNG/TNG50/TNG-50-2-dark/output/'

snapshot_z0 = ill.snapshot.loadSubset(basePath,99,1)  # redshift=0
# snapshot_z10 = ill.snapshot.loadSubset(basePath,4,1)  # redshift=10

In [39]:
# TNG-50-2-hydro
basePath = HOME + '/data/mydata/TNG/TNG50/TNG-50-2-hydro/output/'

snapshot_z0_hydro = ill.snapshot.loadSubset(basePath,99,1)  # redshift=0
# snapshot_z10 = ill.snapshot.loadSubset(basePath,4,1)  # redshift=10

In [17]:
for key in snapshot_z0.keys():
    print(f"{key} shape is {snapshot_z0[key].shape}")
    print(f"{key} len is {len(snapshot_z0[key].shape)}")

count shape is ()
count len is 0
Coordinates shape is (1259712000, 3)
Coordinates len is 2
ParticleIDs shape is (1259712000,)
ParticleIDs len is 1
Potential shape is (1259712000,)
Potential len is 1
SubfindDMDensity shape is (1259712000,)
SubfindDMDensity len is 1
SubfindDensity shape is (1259712000,)
SubfindDensity len is 1
SubfindHsml shape is (1259712000,)
SubfindHsml len is 1
SubfindVelDisp shape is (1259712000,)
SubfindVelDisp len is 1
Velocities shape is (1259712000, 3)
Velocities len is 2


In [40]:
for key in snapshot_z0_hydro.keys():
    print(f"{key} shape is {snapshot_z0_hydro[key].shape}")
    print(f"{key} len is {len(snapshot_z0_hydro[key].shape)}")

count shape is ()
count len is 0
Coordinates shape is (1259712000, 3)
Coordinates len is 2
ParticleIDs shape is (1259712000,)
ParticleIDs len is 1
Potential shape is (1259712000,)
Potential len is 1
SubfindDMDensity shape is (1259712000,)
SubfindDMDensity len is 1
SubfindDensity shape is (1259712000,)
SubfindDensity len is 1
SubfindHsml shape is (1259712000,)
SubfindHsml len is 1
SubfindVelDisp shape is (1259712000,)
SubfindVelDisp len is 1
TimeStep shape is (1259712000,)
TimeStep len is 1
Velocities shape is (1259712000, 3)
Velocities len is 2


In [18]:
# %debug
'''convert dict to modin(pandas)'''
def _dict2pandas(snapshot: dict):
    data = {}
    for key in snapshot.keys():
        if key == 'count':
            print(f"we have {snapshot[key]} particles")
            continue;
        if key=='Coordinates':
            data['x'] = snapshot[key][:,0]
            data['y'] = snapshot[key][:,1]
            data['z'] = snapshot[key][:,2]
        elif key=='Velocities':
            data['vx'] = snapshot[key][:,0]
            data['vy'] = snapshot[key][:,1]
            data['vz'] = snapshot[key][:,2]
        else:
            assert len(snapshot[key].shape)==1
            data[key] = snapshot[key]
    return pd.DataFrame.from_dict(data)
# _dict2pandas({"lalla":np.ones(5),'la':np.random.randint(1,13,5)})
snap_z0 = _dict2pandas(snapshot_z0)
# snap_z10 = _dict2pandas(snapshot_z10)

we have 1259712000 particles


NameError: name 'snapshot_z10' is not defined

In [41]:
# %debug
'''convert dict to modin(pandas)'''
def _dict2pandas(snapshot: dict):
    data = {}
    for key in snapshot.keys():
        if key == 'count':
            print(f"we have {snapshot[key]} particles")
            continue;
        if key=='Coordinates':
            data['x'] = snapshot[key][:,0]
            data['y'] = snapshot[key][:,1]
            data['z'] = snapshot[key][:,2]
        elif key=='Velocities':
            data['vx'] = snapshot[key][:,0]
            data['vy'] = snapshot[key][:,1]
            data['vz'] = snapshot[key][:,2]
        else:
            assert len(snapshot[key].shape)==1
            data[key] = snapshot[key]
    return pd.DataFrame.from_dict(data)
# _dict2pandas({"lalla":np.ones(5),'la':np.random.randint(1,13,5)})
snap_z0_hydro = _dict2pandas(snapshot_z0_hydro)
# snap_z10 = _dict2pandas(snapshot_z10)

we have 1259712000 particles


In [21]:
snap_z0[:7]

,x,y,z,ParticleIDs,Potential,SubfindDMDensity,SubfindDensity,SubfindHsml,SubfindVelDisp,vx,vy,vz
0,7296.924632,24568.351176,21312.822510,292186094,-3514323.75,0.110787,0.110787,0.399294,267.993988,11.656587,11.274169,-16.417538
1,7296.932882,24568.398804,21312.739381,180172719,-3513732.50,0.104651,0.104651,0.399269,259.248810,4.139137,-3.863538,-7.206649
2,7296.684136,24568.498037,21312.993670,292425646,-3512033.00,0.098591,0.098591,0.399294,453.847382,35.386509,-50.640720,-48.297626
3,7296.604182,24568.044952,21312.614194,291646318,-3513162.75,0.076193,0.076193,0.417472,309.247772,12.642206,-49.163681,-88.658333
4,7297.124938,24568.245039,21312.843974,292120622,-3513653.25,0.096166,0.096166,0.379291,353.344879,-31.885572,-22.264559,-52.468033
5,7296.835875,24568.174757,21312.482330,180246703,-3513868.25,0.100623,0.100623,0.417472,335.645905,62.534985,-62.700596,-58.067600
6,7296.434543,24568.586421,21312.892932,192700079,-3510303.75,0.082332,0.082332,0.392792,476.673218,13.941493,10.262628,-61.322060


In [42]:
snap_z0_hydro[:5]

,x,y,z,ParticleIDs,Potential,SubfindDMDensity,SubfindDensity,SubfindHsml,SubfindVelDisp,TimeStep,vx,vy,vz
0,7320.396768,24545.045533,21301.742153,246453549,-4624486.5,0.115510,8.600646,0.316900,1127.373047,0.0,-2.724034,-130.237595,194.834244
1,7320.019628,24545.882536,21301.716311,274619053,-4536184.0,0.099503,3.602955,0.320954,999.061768,0.0,-57.519440,-87.426674,-156.271713
2,7319.812123,24545.252989,21301.910024,246428269,-4547894.5,0.116043,2.392738,0.331315,1017.885559,0.0,11.053613,148.860748,193.462601
3,7320.773122,24545.727954,21301.824647,251882413,-4508384.0,0.097173,2.708374,0.340302,951.596008,0.0,40.403690,146.931046,-175.367508
4,7319.902249,24545.577239,21301.963429,240084397,-4566137.5,0.087756,5.054679,0.325676,998.329468,0.0,-352.747528,-20.196817,92.991341


### save data

In [22]:
import pyarrow.feather as feather

In [24]:
feather.write_feather(snap_z0, "./tmp_save/TNG-50-dark-z0.df.feather")

### load data

In [ ]:
snap_z0 = feather.read_feather("./tmp_save/TNG-50-dark-z0.df.feather")

In [26]:
snap_z0.describe()

,x,y,z,ParticleIDs,Potential,SubfindDMDensity,SubfindDensity,SubfindHsml,SubfindVelDisp,vx,vy,vz
count,1.259712e+09,1.259712e+09,1.259712e+09,1.259712e+09,1.259712e+09,1.259712e+09,1.259712e+09,1.259712e+09,1.259712e+09,1.259712e+09,1.259712e+09,1.259712e+09
mean,1.710434e+04,1.737597e+04,1.702184e+04,6.298560e+08,-2.172204e+05,1.639139e-04,1.639139e-04,3.332785e+01,1.781038e+02,-8.196814e-05,-1.570302e-03,-2.265189e-04
std,1.003194e+04,9.724501e+03,1.002125e+04,3.636475e+08,3.852756e+05,1.218670e-03,1.218670e-03,4.335664e+01,2.159134e+02,2.059004e+02,2.152229e+02,2.176534e+02
min,4.485572e-05,1.684470e-05,1.998037e-05,1.000000e+00,-3.514678e+06,5.639427e-11,5.639427e-11,2.832699e-01,1.293902e+00,-2.361809e+03,-2.282840e+03,-2.423414e+03
25%,7.727775e+03,7.902193e+03,7.429620e+03,3.149280e+08,-2.564365e+05,7.190292e-08,7.190292e-08,6.232785e+00,2.683383e+01,-9.646535e+01,-1.012733e+02,-1.022409e+02
50%,1.754239e+04,1.866426e+04,1.754031e+04,6.298560e+08,-8.575772e+04,1.562124e-06,1.562124e-06,1.485291e+01,9.033804e+01,-4.070946e+00,-4.754377e+00,3.439350e+00
75%,2.497935e+04,2.504672e+04,2.522856e+04,9.447840e+08,-8.217911e+03,2.102140e-05,2.102140e-05,4.243378e+01,2.493005e+02,9.531628e+01,9.940436e+01,1.043707e+02
max,3.500000e+04,3.500000e+04,3.500000e+04,1.259712e+09,8.504098e+04,2.962241e-01,2.962241e-01,3.967981e+02,1.636388e+03,2.234306e+03,2.260423e+03,2.261831e+03


In [33]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_colwidth",1000)
pd.set_option('display.width',1000)

In [34]:
snap_z0['x'].nsmallest(10)

724643068     0.000045
296909490     0.000110
714344894     0.000111
490681069     0.000154
1205429353    0.000157
297436475     0.000163
297796771     0.000176
742589969     0.000200
197453674     0.000208
297789784     0.000218
Name: x, dtype: float64

In [32]:
snap_z0[296909490:296909491]

,x,y,z,ParticleIDs,Potential,SubfindDMDensity,SubfindDensity,SubfindHsml,SubfindVelDisp,vx,vy,vz
296909490,0.00011,5277.34668,21466.23705,1247698445,-152407.34375,0.000024,0.000024,6.388704,310.670898,11.42907,-125.815704,87.781929


In [27]:
import sys
lala = np.empty(int(1024*1024/8))


In [28]:
sys.getsizeof(lala)

1048672

## TNG-50-2 数据描述
[**官方文档datamodel**](https://www.tng-project.org/data/docs/specifications/)   
original box:
> boxsize = 35 Mpc/h = 51.7 Mpc   
> $N_{DM} = 1080^3 = 1,259,712,000$   

划分出的小box，约等于原长 `1/10`
> boxlen = 3.5 Mpc/h   
> $N_{DM} \approx 108^3$

In [ ]:
illustris_python.snapshot.

def loadSubset(basePath, snapNum, partType, fields=None):
    """ Load a subset of fields for all particles/cells of a given partType. """
        
def loadSubhalo(basePath, snapNum, id, partType, fields=None):
    """ Load all particles/cells of one type for a specific subhalo
        (optionally restricted to a subset fields). """
        
def loadHalo(basePath, snapNum, id, partType, fields=None):
    """ Load all particles/cells of one type for a specific halo
        (optionally restricted to a subset fields). """

In [ ]:
# TNG-50-2-dark
basePath = HOME + '/data/mydata/TNG/TNG50/TNG-50-2-dark/output/'

snapshot_z0 = ill.snapshot.loadSubset(basePath,99,1)  # redshift=0
# snapshot_z10 = ill.snapshot.loadSubset(basePath,4,1)  # redshift=10

In [47]:
ill.snapshot.loadSubhalo(HOME + '/data/mydata/TNG/TNG50/TNG-50-2-dark/output/',
                         99,2120000,'dm')

IndexError: Index (1149279) out of range for (0-21206)

In [95]:
# subhaloID: 0~10455
subhaloID = 859077 -1 
ill.snapshot.loadSubhalo(HOME + '/data/mydata/TNG/TNG50/TNG-50-2-hydro/output/',
                         99,subhaloID,'dm')

{'count': 31,
 'Coordinates': array([[34740.19169995, 21072.85366425, 11411.9219204 ],
        [34740.75188847, 21073.84225947, 11409.50707192],
        [34739.68337633, 21072.43337453, 11411.51060939],
        [34739.23222577, 21072.85905529, 11413.11568898],
        [34741.75240612, 21073.86896198, 11410.17342527],
        [34741.37457355, 21074.30658638, 11409.94095933],
        [34740.26086178, 21071.72968923, 11410.09716221],
        [34740.04259457, 21072.11533605, 11413.02699653],
        [34740.54785278, 21071.92629516, 11409.47103851],
        [34741.25864307, 21072.46440508, 11410.6412915 ],
        [34740.4858339 , 21073.78429068, 11413.48206286],
        [34741.12048449, 21073.73449073, 11410.54150034],
        [34738.54684594, 21073.31021613, 11413.96280286],
        [34738.70404017, 21072.76489136, 11412.44051767],
        [34739.41143066, 21070.49692049, 11411.15034834],
        [34740.87260488, 21075.49554239, 11409.84900416],
        [34740.68040027, 21074.26682548, 11

In [57]:
subhalos_z0_hydro = ill.groupcat.loadSubhalos(HOME + '/data/mydata/TNG/TNG50/TNG-50-2-hydro/output/',99)

In [107]:
subhalos_z0_hydro['count']

859077

In [116]:
i = 0
for key in subhalos_z0_hydro.keys():
    print('{} has length: {}'.format(key,np.shape(subhalos_z0_hydro[key])))
    i += 1
    pass
print(i)


count has length: ()
SubhaloBHMass has length: (859077,)
SubhaloBHMdot has length: (859077,)
SubhaloBfldDisk has length: (859077,)
SubhaloBfldHalo has length: (859077,)
SubhaloCM has length: (859077, 3)
SubhaloFlag has length: (859077,)
SubhaloGasMetalFractions has length: (859077, 10)
SubhaloGasMetalFractionsHalfRad has length: (859077, 10)
SubhaloGasMetalFractionsMaxRad has length: (859077, 10)
SubhaloGasMetalFractionsSfr has length: (859077, 10)
SubhaloGasMetalFractionsSfrWeighted has length: (859077, 10)
SubhaloGasMetallicity has length: (859077,)
SubhaloGasMetallicityHalfRad has length: (859077,)
SubhaloGasMetallicityMaxRad has length: (859077,)
SubhaloGasMetallicitySfr has length: (859077,)
SubhaloGasMetallicitySfrWeighted has length: (859077,)
SubhaloGrNr has length: (859077,)
SubhaloHalfmassRad has length: (859077,)
SubhaloHalfmassRadType has length: (859077, 6)
SubhaloIDMostbound has length: (859077,)
SubhaloLen has length: (859077,)
SubhaloLenType has length: (859077, 6)
Subh

### subhalo
1. TNG-50-dark:   
   `0~21206`
2. TNG-50-hydro:   
   `0~859076`  
   
Maybe the **feedback** is too strong, so that the structure of subhalo is
    destroyed!

'''match subhalo(galaxy) properties to each particle
According to particleID, we can allocate all of the 
    galaxies' properties to every particle
save result using feather!
'''
subhalo_tot = 859077
# init
for _, key in subhalos_z0_hydro.keys():
    if len(subhalos_z0_hydro[key])==1:
        exec('{} = {}'.format(key, np.zeros(subhalo_tot)))
    elif len(subhalos_z0_hydro[key])==2:
        dims = subhalos_z0_hydro[key].shape[1]
        for dim in range(1,dims+1):
            exec('{}_{} = {}'.format(key, dim, np.zeros(subhalo_tot)))
    else:
        print(key," Its dimensions is wired!!!")
    pass

# give value to them
for i in range(subhalo_tot):
    subhalos_z0_hydro[]



subhalos_z0_hydro['SubhaloMass'][859076]
# 710, 39

In [117]:
# sort Particles as (x,y,z)
sorted_xyz_snap_z0_hydro = snap_z0_hydro.sort_values(by=['x','y','z'])


In [118]:
sorted_xyz_snap_z0_hydro[:20]

,x,y,z,ParticleIDs,Potential,SubfindDMDensity,SubfindDensity,SubfindHsml,SubfindVelDisp,TimeStep,vx,vy,vz
1230470592,0.000084,3520.735596,26538.051001,1192778054,-4721.208008,1.017204e-08,1.068087e-08,88.680351,21.827280,0.0,91.490761,132.463806,25.315193
713456111,0.000124,12468.872516,12145.237097,6917716,-18012.658203,1.182955e-08,1.334561e-08,82.397766,29.306019,0.0,-65.214706,191.230560,-162.020874
173274444,0.000150,30142.896108,19112.324412,1220339765,-385933.687500,3.402735e-06,3.952722e-06,10.601799,448.895447,0.0,458.361725,-76.121445,372.368744
775641436,0.000185,13569.203374,3522.233370,150269849,-8632.726562,3.215259e-09,3.450142e-09,115.817772,18.148800,0.0,-115.432480,28.921080,22.185621
642233842,0.000192,6547.981955,27290.807414,1175493335,-75504.351562,1.266773e-06,1.266805e-06,14.943611,74.230087,0.0,223.244675,-183.164734,-34.822487
177051935,0.000242,30000.893127,19162.801842,150151109,-355421.968750,2.809508e-06,3.282017e-06,10.602464,439.637848,0.0,162.271149,-544.650696,-572.372864
771099648,0.000258,10318.988442,424.500305,139702616,-62760.699219,8.463393e-07,9.129087e-07,16.832470,110.557915,0.0,-45.481865,57.727371,-20.041666
717093857,0.000278,33428.794542,1992.908601,14943289,52588.480469,6.433537e-08,7.831340e-08,39.130501,30.082819,0.0,-51.181358,54.740261,58.956612
288406211,0.000359,5250.985711,21534.034352,1211100365,-136003.046875,2.264801e-05,2.316770e-05,5.632929,296.923401,0.0,-279.490906,-158.720459,42.946980
174862890,0.000416,30322.929772,19245.144567,67992765,-371268.093750,3.944524e-06,4.567921e-06,10.602464,431.344360,0.0,442.493378,345.324799,-300.950745


In [98]:
sorted_snap_z0_hydro = snap_z0_hydro.sort_values(by=['ParticleIDs'],kind='mergesort')


In [99]:
sorted_snap_z0_hydro[:10]

,x,y,z,ParticleIDs,Potential,SubfindDMDensity,SubfindDensity,SubfindHsml,SubfindVelDisp,TimeStep,vx,vy,vz
488417079,34958.272457,4147.999380,101.031591,1,-21789.867188,9.684982e-05,1.127654e-04,3.548745,96.247559,0.0,-15.303227,179.104767,-52.324615
219655615,30982.062892,33807.400305,7443.342883,2,-773427.437500,3.599016e-03,3.951053e-03,1.133151,546.592224,0.0,-646.540039,553.871155,-110.877686
710488386,31332.894297,32864.700278,7406.670241,3,-81704.507812,4.906107e-08,7.181565e-08,48.549191,125.720963,0.0,-308.757263,51.450310,-91.171288
710488387,33878.513320,29841.257792,15100.289913,4,-90694.023438,3.916100e-07,4.042651e-07,23.340504,70.543877,0.0,89.024529,-182.521225,18.852116
170963638,275.582476,30138.128349,19306.709334,5,-519330.968750,2.550301e-05,2.636746e-05,5.300789,470.803009,0.0,-161.742508,367.678162,-11.784318
176574821,576.556220,29884.399028,19452.777776,6,-286728.531250,4.045803e-05,4.065643e-05,4.631168,125.235840,0.0,252.422424,2.113380,282.750031
459386931,1967.998311,34744.772376,23778.288290,7,-20670.279297,2.101579e-06,2.488119e-06,11.160696,90.881195,0.0,133.516785,-28.977976,-152.783203
710488388,1154.491740,4533.466919,29626.673357,8,-7106.122070,3.717256e-08,4.110987e-08,53.466560,13.950252,0.0,34.703827,201.372696,-71.513351
710488389,34254.516759,4508.942935,34383.311205,9,-409.488983,1.138087e-07,1.159989e-07,28.177691,13.702890,0.0,38.205841,52.647827,-18.620832
710488390,32518.577018,5107.420751,3637.515552,10,-10905.321289,8.843983e-09,1.117425e-08,77.064888,10.256377,0.0,-145.191895,56.121593,-21.077656


In [100]:
sorted_snap_z0_hydro[-10:]

,x,y,z,ParticleIDs,Potential,SubfindDMDensity,SubfindDensity,SubfindHsml,SubfindVelDisp,TimeStep,vx,vy,vz
261023406,31177.589277,23874.028041,32354.044428,1259711991,-119778.218750,2.395784e-06,2.564692e-06,10.601799,262.237793,0.0,112.874779,-161.383362,-6.897445
266829109,29060.170446,21407.331886,574.485480,1259711992,-118687.875000,2.855454e-06,2.937911e-06,11.420728,239.649323,0.0,-53.097401,33.024883,29.592445
223017245,31035.921324,33827.799694,7037.519072,1259711993,-187283.937500,5.735642e-06,5.853378e-06,10.142707,308.388306,0.0,-171.043823,-64.528183,-168.260574
157193969,31767.596568,30329.637521,11682.170082,1259711994,-272820.312500,6.645793e-07,8.023521e-07,17.250059,336.301208,0.0,41.682800,61.496132,94.002190
151461480,32051.179861,30241.288521,12114.025590,1259711995,-938152.937500,6.961754e-04,7.345792e-04,2.103848,667.514526,0.0,-260.672089,-160.337357,315.961426
1259711998,34794.298568,30541.402026,15992.239710,1259711996,-79828.398438,1.501309e-07,1.538436e-07,27.436832,56.295280,0.0,157.368042,-8.883937,-9.681094
176097471,34817.733953,30237.767255,19000.392312,1259711997,-309343.281250,3.309767e-05,3.764257e-05,4.816388,153.488602,0.0,-259.188293,-231.585510,-28.590818
294024751,33620.293479,29355.716380,21731.163431,1259711998,-233317.078125,1.201517e-04,1.497815e-04,3.071176,259.312988,0.0,134.225418,-76.303329,325.336700
535179246,31596.829820,28944.478678,27999.901070,1259711999,33986.386719,1.026435e-06,1.040534e-06,15.292995,27.112312,0.0,79.575180,-82.174866,-97.944946
1259711999,29811.470350,30297.504004,2659.524674,1259712000,27167.353516,3.377796e-07,3.672108e-07,23.056698,59.825199,0.0,-113.551262,-31.858885,113.440529


In [101]:
1080**3

1259712000

# Match particle

## match particle with hydro subhalo catalog
1. 初始化pd.Dataframe表，每一行表示一个DM粒子。第一列命名为ParticleID，和index一样。其余列表示每个粒子所在的subhalo的物理性质。【先使用hydro，检验扩展性时，使用纯DM】
2. 根据`ill.snapshot.loadSubhalo` and `ill.groupcat.loadSubhalos` 读取的数据，进行表中数据的填充。使用multiprocessing，把最外层循环（subhalo）并行，每个subhalo内部的循环处理其粒子即可。
3. 完成particle表之后，可以直接使用pd的merge，根据ParticleID，将其与snap表合并。

In [ ]:
# init 
PARTICLE_NUM = 1080**3
INIT_ARRAY = np.zeros(PARTICLE_NUM)
p_tab = pd.DataFrame()

# give default value of each column
p_tab['ParticleIDs'] = np.arange(1,PARTICLE_NUM+1)
keys_p_tab = [
    'SubhaloGrNr', \
    'SubhaloPos_x','SubhaloPos_y','SubhaloPos_z', 'SubhaloVel_x','SubhaloVel_y','SubhaloVel_z',\
    'DMparNums','SubhaloLen', 'SubhaloMass',  'SubhaloMassInRad', 'SubhaloSFR',\
    'SubhaloSpin_x','SubhaloSpin_y','SubhaloSpin_z',\
    'SubhaloHalfmassRad', 'SubhaloStellarPhotometricsRad','SubhaloVelDisp', 'SubhaloVmax',\
    'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloBfldHalo', \
    'SubhaloGasMetallicity', 'SubhaloStarMetallicity', 'SubhaloGasMetalFractions_H', 'SubhaloStarMetalFractions_H',\
    'SubhaloStellarPhotometrics_U','SubhaloStellarPhotometrics_B','SubhaloStellarPhotometrics_V','SubhaloStellarPhotometrics_K',\
    'SubhaloStellarPhotometrics_g','SubhaloStellarPhotometrics_r','SubhaloStellarPhotometrics_i','SubhaloStellarPhotometrics_z'
]
keys_origin = [
    'SubhaloGrNr', \
    'SubhaloPos', 'SubhaloVel',\
    'DMparNums','SubhaloLen', 'SubhaloMass',  'SubhaloMassInRad', 'SubhaloSFR',\
    'SubhaloSpin',\
    'SubhaloHalfmassRad', 'SubhaloStellarPhotometricsRad','SubhaloVelDisp', 'SubhaloVmax',\
    'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloBfldHalo', \
    'SubhaloGasMetallicity', 'SubhaloStarMetallicity', 'SubhaloGasMetalFractions', 'SubhaloStarMetalFractions',\
    'SubhaloStellarPhotometrics'
]
for key in keys_p_tab:
    p_tab[key] = 0  #INIT_ARRAY

# give value to each particle!
xyz_dict = {0: 'x', 1: 'y', 2: 'z'}
photo_dict = {
    0: 'U', 1: 'B', 2: 'V', 3: 'K',
    4: 'g', 5: 'r', 6: 'i', 7: 'z'
}

# SUBHALO_NUM = subhalos_z0_hydro['count']  # total number of subhalos
SUBHALO_NUM = 10
for subhalo_idx in range(SUBHALO_NUM):
    # exclude SubhaloFlag == 0 [https://www.tng-project.org/data/docs/specifications/#sec2b]
    if subhalos_z0_hydro['SubhaloFlag'][subhalo_idx] == 0: continue
    
    # get particleIDs in this subhalo!
    tmp_subhalo_snap = ill.snapshot.loadSubhalo(HOME + '/data/mydata/TNG/TNG50/TNG-50-2-hydro/output/',99,subhalo_idx,'dm')
    tmp_paridx = tmp_subhalo_snap['ParticleIDs'] - 1
    
    # give value to these particles
    for key in keys_p_tab:
        if key in ['SubhaloGasMetalFractions','SubhaloStarMetalFractions']:
            assert subhalos_z0_hydro[key].value.shape[1] == 10
            p_tab.loc[key + '_H'][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx,0]
            #p_tab.iloc[[tmp_paridx],df.columns.get_loc(key + '_H')]
        elif key in ['SubhaloPos','SubhaloVel','SubhaloSpin']:
            assert subhalos_z0_hydro[key].value.shape[1] == 3
            for i in range(3): p_tab.loc[key + '_' + xyz_dict[i]][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx,i]
        elif key == 'SubhaloStellarPhotometrics':
            assert subhalos_z0_hydro[key].value.shape[1] == 8
            for i in range(8): p_tab.loc[key + '_' + photo_dict[i]][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx,i]
        elif key == 'DMparNums':
            p_tab.loc[key][tmp_paridx] = tmp_subhalo_snap['count']
        else:
            p_tab.loc[key][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx]
        pass
    pass

In [246]:
p_tab.keys()

Index(['ParticleIDs', 'SubhaloGrNr', 'SubhaloPos_x', 'SubhaloPos_y', 'SubhaloPos_z', 'SubhaloVel_x', 'SubhaloVel_y', 'SubhaloVel_z', 'DMparNums', 'SubhaloLen', 'SubhaloMass', 'SubhaloMassInRad', 'SubhaloSFR', 'SubhaloSpin_x', 'SubhaloSpin_y', 'SubhaloSpin_z', 'SubhaloHalfmassRad', 'SubhaloStellarPhotometricsRad', 'SubhaloVelDisp', 'SubhaloVmax', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloBfldHalo', 'SubhaloGasMetallicity', 'SubhaloStarMetallicity', 'SubhaloGasMetalFractions_H', 'SubhaloStarMetalFractions_H', 'SubhaloStellarPhotometrics_U', 'SubhaloStellarPhotometrics_B', 'SubhaloStellarPhotometrics_V', 'SubhaloStellarPhotometrics_K', 'SubhaloStellarPhotometrics_g', 'SubhaloStellarPhotometrics_r', 'SubhaloStellarPhotometrics_i', 'SubhaloStellarPhotometrics_z'], dtype='object')

In [179]:
'''test!!!'''
subhalo_idx = 666
# exclude SubhaloFlag == 0 [https://www.tng-project.org/data/docs/specifications/#sec2b]
if subhalos_z0_hydro['SubhaloFlag'][subhalo_idx] != 0:
    # get particleIDs in this subhalo!
    tmp_subhalo_snap = ill.snapshot.loadSubhalo(HOME + '/data/mydata/TNG/TNG50/TNG-50-2-hydro/output/',99,subhalo_idx,'dm')
    tmp_paridx = tmp_subhalo_snap['ParticleIDs'] - 1
    

In [244]:
p_tab['SubhaloGasMetalFractions' + '_H'][tmp_paridx]

315747883    0.0
315733355    0.0
316558827    0.0
Name: SubhaloGasMetalFractions_H, dtype: float64

In [251]:
subhalos_z0_hydro['SubhaloPos'].shape

(859077, 3)

In [ ]:
keys_origin = [
    'SubhaloGrNr', \
    'SubhaloPos', 'SubhaloVel',\
    'DMparNums','SubhaloLen', 'SubhaloMass',  'SubhaloMassInRad', 'SubhaloSFR',\
    'SubhaloSpin',\
    'SubhaloHalfmassRad', 'SubhaloStellarPhotometricsRad','SubhaloVelDisp', 'SubhaloVmax',\
    'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloBfldHalo', \
    'SubhaloGasMetallicity', 'SubhaloStarMetallicity', 'SubhaloGasMetalFractions', 'SubhaloStarMetalFractions',\
    'SubhaloStellarPhotometrics'
]
# get piece of data of this subhalo!

# give value to these particles
for key in keys_origin:
    if key in ['SubhaloGasMetalFractions','SubhaloStarMetalFractions']:
        assert subhalos_z0_hydro[key].shape[1] == 10
        p_tab[key + '_H'][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx,0]
        #p_tab.iloc[[tmp_paridx],df.columns.get_loc(key + '_H')]
    elif key in ['SubhaloPos','SubhaloVel','SubhaloSpin']:
        assert subhalos_z0_hydro[key].shape[1] == 3
        for i in range(3): p_tab[key + '_' + xyz_dict[i]][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx,i]
    elif key == 'SubhaloStellarPhotometrics':
        assert subhalos_z0_hydro[key].shape[1] == 8
        for i in range(8): p_tab[key + '_' + photo_dict[i]][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx,i]
    elif key == 'DMparNums':
        p_tab[key][tmp_paridx] = tmp_subhalo_snap['count']
    else:
        p_tab[key][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx]
    pass

In [253]:
tmp_paridx

array([315747883, 315733355, 316558827], dtype=uint64)

In [254]:
p_tab.loc[tmp_paridx,:]

,ParticleIDs,SubhaloGrNr,SubhaloPos_x,SubhaloPos_y,SubhaloPos_z,SubhaloVel_x,SubhaloVel_y,SubhaloVel_z,DMparNums,SubhaloLen,SubhaloMass,SubhaloMassInRad,SubhaloSFR,SubhaloSpin_x,SubhaloSpin_y,SubhaloSpin_z,SubhaloHalfmassRad,SubhaloStellarPhotometricsRad,SubhaloVelDisp,SubhaloVmax,SubhaloBHMass,SubhaloBHMdot,SubhaloBfldHalo,SubhaloGasMetallicity,SubhaloStarMetallicity,SubhaloGasMetalFractions_H,SubhaloStarMetalFractions_H,SubhaloStellarPhotometrics_U,SubhaloStellarPhotometrics_B,SubhaloStellarPhotometrics_V,SubhaloStellarPhotometrics_K,SubhaloStellarPhotometrics_g,SubhaloStellarPhotometrics_r,SubhaloStellarPhotometrics_i,SubhaloStellarPhotometrics_z
315747883,315747884,0.0,6728.256348,24113.056641,21160.638672,475.36084,532.059082,954.215149,710.0,712.0,0.174658,0.068432,0.0,2.202053,7.557337,-8.470781,4.017766,0.39,16.199591,31.31683,0.0,0.0,0.0,0.0,0.000192,0.0,0.759677,-8.671708,-8.70383,-9.311282,-11.280831,-9.053692,-9.566896,-9.791246,-9.898157
315733355,315733356,0.0,6728.256348,24113.056641,21160.638672,475.36084,532.059082,954.215149,710.0,712.0,0.174658,0.068432,0.0,2.202053,7.557337,-8.470781,4.017766,0.39,16.199591,31.31683,0.0,0.0,0.0,0.0,0.000192,0.0,0.759677,-8.671708,-8.70383,-9.311282,-11.280831,-9.053692,-9.566896,-9.791246,-9.898157
316558827,316558828,0.0,6728.256348,24113.056641,21160.638672,475.36084,532.059082,954.215149,710.0,712.0,0.174658,0.068432,0.0,2.202053,7.557337,-8.470781,4.017766,0.39,16.199591,31.31683,0.0,0.0,0.0,0.0,0.000192,0.0,0.759677,-8.671708,-8.70383,-9.311282,-11.280831,-9.053692,-9.566896,-9.791246,-9.898157


In [ ]:
p_tab.loc[tmp_paridx,:]

In [ ]:
# save data into disk
feather.write_feather(p_tab, "./tmp_save/TNG-50-hydro-z0-particle.df.feather")

In [ ]:
snap_z0 = feather.read_feather("./tmp_save/TNG-50-dark-z0.df.feather")
feather.write_feather(snap_z0, "./tmp_save/TNG-50-dark-z0.df.feather")

## match particle (parallel)

In [112]:
'''parallel with multiprocessing!'''
# init 
PARTICLE_NUM = 1080**3
INIT_ARRAY = np.zeros(PARTICLE_NUM)
p_tab = pd.DataFrame()

# give default value of each column
p_tab['ParticleIDs'] = np.arange(1,PARTICLE_NUM+1)
keys_p_tab = [
    'SubhaloGrNr', \
    'SubhaloPos_x','SubhaloPos_y','SubhaloPos_z', 'SubhaloVel_x','SubhaloVel_y','SubhaloVel_z',\
    'DMparNums','SubhaloLen', 'SubhaloMass',  'SubhaloMassInRad', 'SubhaloSFR',\
    'SubhaloSpin_x','SubhaloSpin_y','SubhaloSpin_z',\
    'SubhaloHalfmassRad', 'SubhaloStellarPhotometricsRad','SubhaloVelDisp', 'SubhaloVmax',\
    'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloBfldHalo', \
    'SubhaloGasMetallicity', 'SubhaloStarMetallicity', 'SubhaloGasMetalFractions_H', 'SubhaloStarMetalFractions_H',\
    'SubhaloStellarPhotometrics_U','SubhaloStellarPhotometrics_B','SubhaloStellarPhotometrics_V','SubhaloStellarPhotometrics_K',\
    'SubhaloStellarPhotometrics_g','SubhaloStellarPhotometrics_r','SubhaloStellarPhotometrics_i','SubhaloStellarPhotometrics_z'
]
for key in keys_p_tab:
    p_tab[key] = INIT_ARRAY

# give value to each particle!
xyz_dict = {0: 'x', 1: 'y', 2: 'z'}
photo_dict = {
    0: 'U', 1: 'B', 2: 'V', 3: 'K',
    4: 'g', 5: 'r', 6: 'i', 7: 'z'
}

def _par_match(subhalo_idx):
    
    # exclude SubhaloFlag == 0 [https://www.tng-project.org/data/docs/specifications/#sec2b]
    if subhalos_z0_hydro['SubhaloFlag'][subhalo_idx] == 0: return
    
    # get particleIDs in this subhalo!
    tmp_subhalo_snap = ill.snapshot.loadSubhalo(HOME + '/data/mydata/TNG/TNG50/TNG-50-2-hydro/output/',99,subhalo_idx,'dm')
    tmp_paridx = tmp_subhalo_snap['ParticleIDs'] - 1
    
    # give value to these particles
    for key in keys_p_tab:
        if key in ['SubhaloGasMetalFractions','SubhaloStarMetalFractions']:
            assert subhalos_z0_hydro[key].value.shape[1] == 10
            p_tab.loc[key + '_H'][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx,0]
        elif key in ['SubhaloPos','SubhaloVel','SubhaloSpin']:
            assert subhalos_z0_hydro[key].value.shape[1] == 3
            [p_tab.loc[key + '_' + xyz_dict[i]][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx,i] for i in range(3)]
        elif key == 'SubhaloStellarPhotometrics':
            assert subhalos_z0_hydro[key].value.shape[1] == 8
            [p_tab.loc[key + '_' + photo_dict[i]][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx,i] for i in range(8)]
        else:
            p_tab.loc[key][tmp_paridx] = subhalos_z0_hydro[key][subhalo_idx]
        pass
    
    return


array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [8]:
#************************************************************************************
# start multi-run
import warnings
from multiprocessing import Pool
from time import time
warnings.filterwarnings('ignore')    # Do Not see warnings

pool = Pool(processes=10)
time_start = time()

snapshot_data = list(pool.map(
    _par_match,
    range(SUBHALO_NUM)
))

pool.close() # shut down the process
pool.join()
print(f"Done in {time() - time_start} second")
#************************************************************************************

Done in 2889.538347005844 second


# Divide into little box
根据3.5Mpc划分数据   
particle的`xyz`已经**排序**过，直接按照 *little box* 大小进行划分即可

In [137]:
sort_parID_snap_hydro_z0 = sorted_snap_z0_hydro.reset_index(drop=True)
sort_parID_snap_hydro_z0[:10]

,x,y,z,ParticleIDs,Potential,SubfindDMDensity,SubfindDensity,SubfindHsml,SubfindVelDisp,TimeStep,vx,vy,vz
0,34958.272457,4147.999380,101.031591,1,-21789.867188,9.684982e-05,1.127654e-04,3.548745,96.247559,0.0,-15.303227,179.104767,-52.324615
1,30982.062892,33807.400305,7443.342883,2,-773427.437500,3.599016e-03,3.951053e-03,1.133151,546.592224,0.0,-646.540039,553.871155,-110.877686
2,31332.894297,32864.700278,7406.670241,3,-81704.507812,4.906107e-08,7.181565e-08,48.549191,125.720963,0.0,-308.757263,51.450310,-91.171288
3,33878.513320,29841.257792,15100.289913,4,-90694.023438,3.916100e-07,4.042651e-07,23.340504,70.543877,0.0,89.024529,-182.521225,18.852116
4,275.582476,30138.128349,19306.709334,5,-519330.968750,2.550301e-05,2.636746e-05,5.300789,470.803009,0.0,-161.742508,367.678162,-11.784318
5,576.556220,29884.399028,19452.777776,6,-286728.531250,4.045803e-05,4.065643e-05,4.631168,125.235840,0.0,252.422424,2.113380,282.750031
6,1967.998311,34744.772376,23778.288290,7,-20670.279297,2.101579e-06,2.488119e-06,11.160696,90.881195,0.0,133.516785,-28.977976,-152.783203
7,1154.491740,4533.466919,29626.673357,8,-7106.122070,3.717256e-08,4.110987e-08,53.466560,13.950252,0.0,34.703827,201.372696,-71.513351
8,34254.516759,4508.942935,34383.311205,9,-409.488983,1.138087e-07,1.159989e-07,28.177691,13.702890,0.0,38.205841,52.647827,-18.620832
9,32518.577018,5107.420751,3637.515552,10,-10905.321289,8.843983e-09,1.117425e-08,77.064888,10.256377,0.0,-145.191895,56.121593,-21.077656


In [138]:
# save data into disk
feather.write_feather(sort_parID_snap_hydro_z0, "./tmp_save/TNG-50-hydro-z0-snap-sort-parID.df.feather")

In [ ]:
'''sort by x,y,z'''
%time sort_x_snap_hydro_z0 = sort_parID_snap_hydro_z0.sort_values(by=['x'],kind='mergesort')
%time sort_y_snap_hydro_z0 = sort_parID_snap_hydro_z0.sort_values(by=['y'],kind='mergesort')
%time sort_z_snap_hydro_z0 = sort_parID_snap_hydro_z0.sort_values(by=['z'],kind='mergesort')


In [148]:
def _sort_xyz(pos):
    assert pos in ['x','y','z']
    sorted_data = sort_parID_snap_hydro_z0.sort_values(by=[pos],kind='mergesort')
    return sorted_data

In [ ]:
#************************************************************************************
# start multi-run
import warnings
from multiprocessing import Pool
from time import time
warnings.filterwarnings('ignore')    # Do Not see warnings

pool = Pool(processes=3)
time_start = time()

sorted_data = list(pool.map(
    _sort_xyz,
    ['x','y','z']
))

pool.close() # shut down the process
pool.join()
print(f"Done in {time() - time_start} second")
#************************************************************************************

In [151]:
# lalala
np.shape(sorted_data)

(3, 1259712000, 13)

In [152]:
def _save_sorted_xyz(pos):
    assert pos in ['x','y','z']
    xyz_dict = {'x':0, 'y':1, 'z':2}
    tmp_data = sorted_data[xyz_dict[pos]]
    # save it!
    feather.write_feather(sorted_data[xyz_dict[pos]], './tmp_save/TNG-50-hydro-z0-snap-sorted_' + pos + '.df.feather')
    return

In [153]:
#************************************************************************************
# start multi-run
import warnings
from multiprocessing import Pool
from time import time
warnings.filterwarnings('ignore')    # Do Not see warnings

pool = Pool(processes=3)
time_start = time()

_tmp = list(pool.map(
    _save_sorted_xyz,
    ['x','y','z']
))

pool.close() # shut down the process
pool.join()
print(f"Done in {time() - time_start} second")
#************************************************************************************

Done in 331.96237874031067 second


In [ ]:
sort_x_snap_hydro_z0, sort_y_snap_hydro_z0, sort_z_snap_hydro_z0 = sorted_data

In [ ]:
# save data into disk
feather.write_feather(sorted_xyz_snap_z0_hydro, "./tmp_save/TNG-50-hydro-z0-snap-sorted_xyz.df.feather")

In [ ]:
from itertools import product

h = 0.6774  # Hubble constant/100 [Planck 2015] [https://www.tng-project.org/data/docs/background/#subhaloflag]
mini_boxsize = 64  # grid number per mini-box

# init 3D (x,y,z) box edges (right edge, bigger one)
edge1D = np.arange(64,640+1,64)  # divide Big box into 10^3 mini boxes
edge3D = np.array(list(product(edge1D,repeat=3)))
left_edge3D = (edge3D-1) * ( (50 * 1000 * h) / (mini_boxsize*10) )  # edge * (50Mpc / boxsize) [kpc/h]
right_edge3D = edge3D * ( (50 * 1000 * h) / (mini_boxsize*10) )  # edge * (50Mpc / boxsize) [kpc/h]
pos3D = np.array(list(product(np.arange(1,64+1),repeat=3)))

# give value into each mini-box
keys_p_tab = [
    'SubhaloGrNr', \
    'SubhaloPos_x','SubhaloPos_y','SubhaloPos_z', 'SubhaloVel_x','SubhaloVel_y','SubhaloVel_z',\
    'DMparNums','SubhaloLen', 'SubhaloMass',  'SubhaloMassInRad', 'SubhaloSFR',\
    'SubhaloSpin_x','SubhaloSpin_y','SubhaloSpin_z',\
    'SubhaloHalfmassRad', 'SubhaloStellarPhotometricsRad','SubhaloVelDisp', 'SubhaloVmax',\
    'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloBfldHalo', \
    'SubhaloGasMetallicity', 'SubhaloStarMetallicity', 'SubhaloGasMetalFractions_H', 'SubhaloStarMetalFractions_H',\
    'SubhaloStellarPhotometrics_U','SubhaloStellarPhotometrics_B','SubhaloStellarPhotometrics_V','SubhaloStellarPhotometrics_K',\
    'SubhaloStellarPhotometrics_g','SubhaloStellarPhotometrics_r','SubhaloStellarPhotometrics_i','SubhaloStellarPhotometrics_z'
]

par_x = sorted_x_snap_z0_hydro['x'].values
par_y = sorted_y_snap_z0_hydro['y'].values
par_z = sorted_z_snap_z0_hydro['z'].values

for (xl, yl, zl), (xr, yr, zr) in zip(left_edge3D,right_edge3D):
    # init mini-box
    minibox = pd.Dataframe()
    miniboxID = str(xr)+'_'+str(yr)+'_'+str(zr)
    minibox['boxID'] = [miniboxID] * mini_boxsize**3
    minibox['x'],minibox['y'],minibox['z']=np.hsplit(pos3D,3)
    [minibox[key] = 0 for key in keys_p_tab];
    
    # select particles according to [x_left, x_right)
    idx_xl = np.argmax(par_x>=xl)
    idx_xr = np.argmax(par_x>xr)
    idx_yl = np.argmax(par_y>=yl)
    idx_yr = np.argmax(par_y>yr)
    idx_zl = np.argmax(par_z>=zl)
    idx_zr = np.argmax(par_z>zr)
    
    # get selected particleID
    parID_x = 
    
    pos['z'][pos['z']>0].argmax()
    x_right = par_x[x_pre:].argmax(>x_edge)
    y_right = par_x[y_pre:].argmax(>y_edge)
    z_right = par_x[z_pre:].argmax(>z_edge)
    
    
    
    # get selected particles
    sel_parID = np.intersect1d(ar1, ar2, assume_unique=True)
    sel_par = sorted_snap_z0_hydro.iloc[tmp_parID-1]
    
    sel_par = sorted_xyz_snap_z0_hydro[max(x_pre, y_pre, z_pre):min(x_right,y_right,z_right)]
    
    x_pre, y_pre, z_pre = x_right, y_right, z_right
    
    sorted_snap_z0_hydro
     = sorted_xyz_snap_z0_hydro['x']
    position = np.argmax(array > 5)

In [ ]:
sorted_xyz_snap_z0_hydro

np.argmax(aa>5)
from itertools import product

REALSIZE = 50 # Mpc
mini_boxsize = 64  # grid

pos=list(np.arange(sample_boxsize,2000,1))
ranges=list(product(pos,repeat=3))



In [ ]:
'''smooth the whole box
We can use NGP/CIC to downsample, but for simplicity, we use average and max 
in each grid first!
For each grid, we calculate mass, velocity, SFR, stellar mass, halo mass,
    blackhole mass, metallicity etc.
'''


# Data to mesh
- 对所有特性进行平均处理【平滑】【分配至mesh】
- 若要使用Max，Min，则需要修改pmesh源码，添加一个方法【之后再做】

In [ ]:
from nbodykit.source.catalog import ArrayCatalog

f = ArrayCatalog({'Position' : data['Position'], 'Mass' : data['Mass'] })
试试pandas格式，若不行，则 df 转 dict

https://rainwoodman.github.io/pmesh/_modules/pmesh/window.html#FindResampler
        NEAREST = ResampleWindow(kind="nearest"),
    LINEAR = ResampleWindow(kind="linear"),
    NNB = ResampleWindow(kind="tunednnb"),
    CIC = ResampleWindow(kind="tunedcic"),
    TSC = ResampleWindow(kind="tunedtsc"),
    PCS = ResampleWindow(kind="tunedpcs"),

In [ ]:
# read data
subhalos_z0_hydro = ill.groupcat.loadSubhalos(HOME + '/data/mydata/TNG/TNG50/TNG-50-2-hydro/output/',99)

In [ ]:
import nbodykit